In [1]:
import pandas as pd

In [2]:
# Upload de Datasets
users_ff = pd.read_csv('users_ff.csv')
users_ss1 = pd.read_csv('users_ss1.csv')
users_ss2 = pd.read_csv('users_ss2.csv')
users_ss3 = pd.read_csv('users_ss3.csv')
users_ts = pd.read_csv('users_ts.csv')
tweets_ff = pd.read_csv('tweets_ff.csv')
tweets_ss1 = pd.read_csv('tweets_ss1.csv')
tweets_ss2 = pd.read_csv('tweets_ss2.csv')
tweets_ss3 = pd.read_csv('tweets_ss3.csv')
tweets_ts = pd.read_csv('tweets_ts.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Dataset Usuarios

In [9]:
# Agregar columna para categorizar
users_ss1['Cat'] = 'SocialSpambot'
users_ss2['Cat'] = 'SocialSpambot'
users_ss3['Cat'] = 'SocialSpambot'
users_ts['Cat'] = 'TraditionalSpambot'
users_ff['Cat'] = 'FollowersBot'
# Concatenar datasets de Usuarios (SocialSpambots & Traditional Spambots)
users_spam = pd.concat([users_ss1,users_ss2,users_ss3,users_ts,users_ff],ignore_index=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [ ]:
users_spam.info()

In [13]:
# Exportar a pkl
# users_spam.to_pickle('users_spam.pkl')

### Dataset Tweets

In [11]:
# Agregar columna para categorizar
tweets_ss1['Cat'] = 'SocialSpambot'
tweets_ss2['Cat'] = 'SocialSpambot'
tweets_ss3['Cat'] = 'SocialSpambot'
tweets_ts['Cat'] = 'TraditionalSpambot'
tweets_ff['Cat'] = 'FollowersBot'
# Concatenar datasets de Usuarios (SocialSpambots & Traditional Spambots)
tweets_spam = pd.concat([tweets_ss1,tweets_ss2,tweets_ss3,tweets_ts,tweets_ff],ignore_index=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [12]:
tweets_spam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798254 entries, 0 to 3798253
Data columns (total 26 columns):
Cat                        object
contributors               float64
crawled_at                 object
created_at                 object
favorite_count             int64
favorited                  float64
geo                        float64
id                         int64
in_reply_to_screen_name    object
in_reply_to_status_id      int64
in_reply_to_user_id        int64
num_hashtags               int64
num_mentions               int64
num_urls                   int64
place                      object
possibly_sensitive         float64
reply_count                int64
retweet_count              int64
retweeted                  float64
retweeted_status_id        float64
source                     object
text                       object
timestamp                  object
truncated                  float64
updated                    object
user_id                    int64
dtypes

In [14]:
# Exportar a pkl
# tweets_spam.to_pickle('tweets_spam.pkl')